In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

In [2]:
path = '/chromedriver'
service = Service(executable_path=path)

# Configuración del driver para Google Chrome
# driver = webdriver.Chrome(service=service)

# Configuración del driver para Brave
option = webdriver.ChromeOptions()
option.binary_location = 'C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe'
driver = webdriver.Chrome(service=service, options=option)

The version of chrome cannot be detected. Trying with latest driver version


In [3]:
def get_matches(year):
    # Eligiendo la página para leer mediante el driver
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    driver.get(web)
    
    matches = driver.find_elements(by='xpath', value='//table[@class="fevent"]')

    home = []
    away = []
    score = []
    for match in matches:
        # En la funcion find_element, el value='.' indica que estamos buscando un elemento en un nodo ya creado
        # home = match.find_element(by='xpath', value='.//th[@class="fhome"]') A modo de ejemplo, esta vez usaremos [1] para hacer referencia al equipo local
        home.append(match.find_element(by='xpath', value='.//th[1]').text)
        away.append(match.find_element(by='xpath', value='.//th[3]').text)
        score.append(match.find_element(by='xpath', value='.//th[2]').text)

    dict_football = {'home': home, 'away': away, 'score': score}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    time.sleep(2) # se agrega un tiempo de pausa para que no detecten que hacemos web scraping
    
    return df_football

In [4]:
# Todos los años en los que hubo mundial
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018]

In [5]:
fifa = [get_matches(year) for year in years] # Lista de comprensión
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_worldcup_historical_data.csv', index=False)

driver.quit()

**Nota:** Podemos buscar los elementos mediante su "XPath" en el código html de una página si presionamos "ctrl+F" al abrir la ventana de inspeccionar elemento.

Para el caso de las tablas de resultados de los grupo de mundial el XPath actual es: //th[@class="fhome"]
donde podemos reemplazar la clase "fhome" por "faway" y "fscore" para el equipo local, el equipo visitante y el puntaje del partido.

Adicionalmente, para encontrar el elemento padre de cualquier XPath utilizamos "/..", por ejemplo: //th[@class="fhome"]/..

**En la función find_element de Selenium**, para buscar un elemento dentro de un nodo ya creado, en este caso dentro del bucle "for matches", es necesario poner value='.' antes del XPath del elemento